#### **利用ogr开源库进行矢量文件的读写**
备注:该程序在gdal=3.6.2版本下运行

In [13]:
from osgeo import ogr
import os

##### **1. 基于ogr的矢量文件读入**

In [14]:
path_kunming = 'data/Section-2/kunming_wgs84.gpkg'



In [15]:
### 读入矢量文件
ds = ogr.Open(path_kunming, 0)    # 0是只读，1是可写
print(ds)
layer = ds.GetLayer(0)            # 获取矢量文件Layer
print(layer)
num_fea = layer.GetFeatureCount()   ## 统计要素个数
print('Number of feature: ', num_fea)
fea = layer.GetFeature(1)    ### 获取要素。!!!索引从1开始。
print(fea)
num_field = fea.GetFieldCount()   ## 统计属性个数
print('Number of fields:', num_field)


<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x0000024FCEA5B9C0> >
<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x0000024FCEE77B40> >
Number of feature:  14
<osgeo.ogr.Feature; proxy of <Swig Object of type 'OGRFeatureShadow *' at 0x0000024FCEE77F60> >
Number of fields: 2


##### **1.1. 查看要素属性信息**

In [16]:
### 查看属性字段名
fields = []
fea = layer.GetFeature(1)
for i in range(0, num_field):   ### range
    field = fea.GetFieldDefnRef(i).GetName()   ### 获取字段定义，索引从0开始。
    fields.append(field)
print(fields)


['dt_adcode', 'dt_name']


In [17]:
### 查看属性字段值（以dt_name为例）
field_name = 'dt_name'
fea = layer.GetFeature(1)
print('Field value:', fea.GetField(field_name))
### 获得所有要素的字段值
for i in range(layer.GetFeatureCount()):   ##
  fea = layer.GetFeature(i+1)   ## 索引从1开始。
  print(f'Field value of the feature {i+1}:', fea.GetField(field_name))



Field value: wuhua
Field value of the feature 1: wuhua
Field value of the feature 2: panglong
Field value of the feature 3: guandu
Field value of the feature 4: xishan
Field value of the feature 5: dongchuan
Field value of the feature 6: chenggong
Field value of the feature 7: jinling
Field value of the feature 8: fumin
Field value of the feature 9: yiliang
Field value of the feature 10: shilin
Field value of the feature 11: songming
Field value of the feature 12: luquan
Field value of the feature 13: xuandian
Field value of the feature 14: anling


##### **1.2. 查看矢量文件几何信息**

In [18]:
### 空间参考查看
SpaRef = layer.GetSpatialRef()
SpaRef.ExportToWkt()


'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'

In [19]:
### 几何类型查看
fea_geo = fea.GetGeometryRef()
fea_geo_name = fea_geo.GetGeometryName()
print(fea_geo_name)


MULTIPOLYGON


In [20]:
ds = None   ### 关闭数据源


##### **2. 写出矢量文件**
1. 写出只含某一字段的矢量数据；
2. 写出只含某一要素的矢量数据。

In [21]:
path_kunming_out_1 = 'data/Section-4/kunming_districts_dtname.gpkg'
path_kunming_out_2 = 'data/Section-4/kunming_chenggong.gpkg'


##### 2.1 写出只含有某一字段的矢量文件
1）读入矢量数据并获取矢量数据图层；  
2）创建新矢量文件，依次创建数据源、图层、字段。其中新建图层需定义图层名、几何、空间参考。新建字段需先定义字段，定义字段需提供字段名、字段数据类型。  
3）遍历矢量图层要素，写入要素几何及所需属性字段。


In [22]:
## 读入矢量图层
field_name = 'dt_name'
in_ds = ogr.Open(path_kunming, 0)        # 0是只读，1是可写
in_layer = in_ds.GetLayer(0)             # 获取矢量文件图层Layer
## 定义写出文件
driver = ogr.GetDriverByName('GPKG')   ## 定义文件驱动（用于写出文件,其他: ‘ESRI Shapefile’）
ds_out = driver.CreateDataSource(path_kunming_out_1)  ## 创建数据源DataSource
layer_out = ds_out.CreateLayer('kunming_districts_dtname', \
                                geom_type = ogr.wkbPolygon, \
                                srs = in_layer.GetSpatialRef())
field_defn = ogr.FieldDefn(field_name, ogr.OFTString)   ## 定义属性字段
layer_out.CreateField(field_defn)    ## 在图层中写入属性字段
# fea_defn = layer_out.GetFeature(1).GetDefnRef()    ## layer_out没有要素，故.GetFeature(1)会报错
fea_defn = layer_out.GetLayerDefn()   ## 获得要素定义（即图层定义, 该矢量此时只定义了图层，未定义要素，所以只能通过图层获取）
### 创建要素（将复制‘昆明市边界_wgs84.shp’文件中要素)
for i in range(in_layer.GetFeatureCount()):   ## 遍历各要素
    in_fea = in_layer.GetFeature(i+1)
    in_geo = in_fea.geometry()             ## 获得原矢量要素的几何
    fea_out = ogr.Feature(fea_defn)    ## 创建要素
    fea_out.SetGeometry(in_geo)   ## 对要素写入几何
    field_value = in_fea.GetField(field_name)   ## 从元素要素中获取字段值
    print(field_value)
    fea_out.SetField(0, field_value)
    layer_out.CreateFeature(fea_out)
in_ds = None
ds_out = None         #  ### 保存/关闭 DataSource (!重要)
# os.remove(path_kunming_out_1)  ## 注意创建路径不能被占用


wuhua
panglong
guandu
xishan
dongchuan
chenggong
jinling
fumin
yiliang
shilin
songming
luquan
xuandian
anling


##### 2.2 写出只含有呈贡区的矢量文件


In [23]:
## 读入矢量图层
in_ds = ogr.Open(path_kunming, 0)           # 0是只读，1是可写
in_layer = in_ds.GetLayer()    # 获取矢量文件Layer
### 创建写出文件
driver = ogr.GetDriverByName('GPKG')
ds_out = driver.CreateDataSource(path_kunming_out_2)
layer_out = ds_out.CreateLayer('kunming_chenggong', geom_type=ogr.wkbPolygon, srs=in_layer.GetSpatialRef())
fea_defn = layer_out.GetLayerDefn()      ## 获得空要素定义(或空图层定义)
### 创建字段
# layer_defn = layer_out.GetLayerDefn()    ## 获取图层定义，用于测试新建图层字段数
# print('Number of fields (before field creating): ', layer_defn.GetFieldCount())  ### 字段创建前字段数
in_fea = in_layer.GetFeature(1)   ## 获取要素
for i in range(in_fea.GetFieldCount()):
  field_defn = in_fea.GetFieldDefnRef(i)   ## 创建字段定义
  layer_out.CreateField(field_defn)        ## 向图层写入字段定义
# layer_defn = layer_out.GetLayerDefn()    ## 获取图层定义，用于测试写入字段后字段数
# print('Number of fields (after field creating): ', layer_defn.GetFieldCount())

### 创建要素（复制‘昆明市边界_wgs84.shp’文件中要素)
for i in range(in_layer.GetFeatureCount()):   ## 遍历要素
  in_fea = in_layer.GetFeature(i+1)
  if str(in_fea.GetField('dt_name')) == 'chenggong':
    fea_out = ogr.Feature(fea_defn)
    fea_out.SetGeometry(in_fea.geometry())
    for i in range(in_fea.GetFieldCount()):   ## 遍历字段
      field_value = in_fea.GetField(i)
      fea_out.SetField(i, field_value)
    layer_out.CreateFeature(fea_out)   ## 将要素写入到新建图层中
in_ds = None
ds_out = None        #  ### 保存/关闭 DataSource (重要)


##### 课后练习：写出只含五华区、盘龙区、西山区、官渡区、呈贡区的矢量文件。